In [ ]:
# ============================================================
# CREDIT RISK & COLLECTIONS MODEL
# Author: Andrea Rodríguez
# Description:
# End-to-end pipeline for credit collections risk modeling
# using temporal validation and segmentation by delinquency bucket.
# ============================================================


# ===============================
# LIBRERÍAS
# ===============================

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    confusion_matrix,
    classification_report,
    roc_curve
)
from lightgbm import LGBMClassifier


# ===============================
# CARGA DE DATOS
# ===============================

# Directorio base
user_dir = "***"

# Fuentes operativas
cliente = pd.read_csv(os.path.join(user_dir, "***"))
econ    = pd.read_csv(os.path.join(user_dir, "***"))
prod    = pd.read_csv(os.path.join(user_dir, '***'))
com     = pd.read_csv(os.path.join(user_dir, '***'))
pago    = pd.read_csv(os.path.join(user_dir, '***'))
ree     = pd.read_csv(os.path.join(user_dir, '***'))
cas     = pd.read_csv(os.path.join(user_dir, '***'))
ges     = pd.read_csv(os.path.join(user_dir, '***'))


# ===============================
# FUNCIONES AUXILIARES
# ===============================

def convertir_columnas_fecha(df):
    """
    Convierte todas las columnas cuyo nombre inicia con 'FECHA'
    a tipo datetime.
    """
    for col in df.columns:
        if col.upper().startswith("FECHA"):
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
    return df


def analizar_serie_temporal(df, fecha_col, valor_col, freq='M'):
    """
    Descomposición temporal para analizar tendencia y estacionalidad.
    """
    df = df[[fecha_col, valor_col]].dropna()
    df = df.set_index(fecha_col).resample(freq).sum()

    descomposicion = seasonal_decompose(df, model='additive')
    descomposicion.plot()
    plt.suptitle(f'{fecha_col} - Descomposición temporal')
    plt.tight_layout()
    plt.show()


def correlaciones(df, columnas_numericas):
    """
    Heatmap de correlaciones entre variables numéricas.
    """
    corr = df[columnas_numericas].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Matriz de correlación")
    plt.show()


# ===============================
# ANÁLISIS EXPLORATORIO (Ejemplo de Uso)
# ===============================

cliente.info()
convertir_columnas_fecha(cliente)

# Edad del cliente
cliente['Edad'] = pd.to_datetime('today').year - cliente['FECHA NACIMIENTO'].dt.year

sns.countplot(x='SEXO', data=cliente)
plt.title('Distribución por sexo')
plt.show()

cliente['Edad'].hist(bins=90)
plt.title('Distribución de Edad')
plt.xlabel('Edad')
plt.xlim(0, 100)
plt.ylabel('Frecuencia')
plt.show()

# Altura mora
com['ALTURA DE MORA'].hist(bins=50)
plt.title('Distribución de Mora')
plt.xlabel('Mora')
plt.xlim(0, 200)
plt.ylabel('Frecuencia')
plt.show()

# Outliers
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
plt.suptitle('Boxplots: Outliers en variables sociodemográficas')

sns.boxplot(y=cliente['NEGOCIO'], ax=axes[0,0])
sns.boxplot(y=cliente['DEPENDIENTES ECONOMICOS'], ax=axes[0,1])
sns.boxplot(y=cliente['TIEMPO VIVIENDA ANIOS'], ax=axes[1,0])
sns.boxplot(y=cliente['TIEMPO VIVIENDA MESES'], ax=axes[1,1])

axes[1,0].set_ylabel("TIEMPO VIVIENDA AÑOS")
plt.tight_layout()
plt.show()

# Serie temporal de vinculaciones
analizar_serie_temporal(cliente, 'FECHA VINCULACION', 'FOLADE', freq='M')

# Correlación
convertir_columnas_fecha(ree)
columnas_correlacion = ree.columns.tolist()
correlaciones(ree, columnas_correlacion)



In [ ]:
# ===============================
# CONSTRUCCIÓN DEL DATASET MODELABLE
# ===============================

df = com.copy()

df['FECHA_CORTE_BASE'] = pd.to_datetime(df['FECHA CORTE BASE'])
df['FECHA_ULTIMO_PAGO'] = pd.to_datetime(df['FECHA ULTIMO PAGO'], errors='coerce')

df = df.sort_values(['FOLADE', 'NEGOCIO', 'FECHA_CORTE_BASE'])

# Mora actual y futura
df['mora_t']  = df['ALTURA DE MORA']
df['mora_t1'] = df.groupby(['FOLADE','NEGOCIO'])['ALTURA DE MORA'].shift(-1)


def left_join(base, other):
    return base.merge(other, how='left', on=['FOLADE','NEGOCIO'])


df = left_join(df, cliente)
df = left_join(df, econ)
df = left_join(df, prod)


# ===============================
# FEATURES DE PAGOS
# ===============================

pago['MES_PAGO'] = pd.to_datetime(pago['MES_PAGO'])

pagos_agg = (
    pago.groupby(['FOLADE','NEGOCIO'])
         .agg(
             pago_total=('PAGO_TOTAL','sum'),
             num_pagos=('PAGO_TOTAL','count')
         )
         .reset_index()
)

df = df.merge(pagos_agg, how='left', on=['FOLADE','NEGOCIO'])
df[['pago_total','num_pagos']] = df[['pago_total','num_pagos']].fillna(0)

df['ratio_pago_saldo'] = df['pago_total'] / (df['SALDO TOTAL'] + 1)


# ===============================
# FEATURES DE GESTIÓN
# ===============================

# En la base gestion hay espacios en los nombre de las columnas
#ges['          FECHA CORTE']
ges.columns = ges.columns.str.strip()

ges['FECHA CORTE'] = pd.to_datetime(ges['FECHA CORTE'])

ges_agg = (
    ges.groupby(['FOLADE','NEGOCIO'])
       .agg(
           num_gestiones=('TIPO GESTION','count'),
           contactabilidad=('RESULTADO', lambda x: np.mean(x=='CONTACTO'))
       )
       .reset_index()
)

df = df.merge(ges_agg, how='left', on=['FOLADE','NEGOCIO'])
df[['num_gestiones','contactabilidad']] = df[['num_gestiones','contactabilidad']].fillna(0)


# ===============================
# EVENTOS DE RIESGO FUTURO (TARGET)
# ===============================

df['fin_mes_t1'] = df['FECHA_CORTE_BASE'] + pd.offsets.MonthEnd(1)

cas['FECHA_CASTIGO'] = pd.to_datetime(cas['FECHA CASTIGO'])

df = df.merge(
    cas[['FOLADE','NEGOCIO','FECHA_CASTIGO']],
    how='left',
    on=['FOLADE','NEGOCIO']
)

df['castigo_t1'] = (
    (df['FECHA_CASTIGO'] > df['FECHA_CORTE_BASE']) &
    (df['FECHA_CASTIGO'] <= df['fin_mes_t1'])
)

ree['FECHA_REESTRUCTURACION'] = pd.to_datetime(ree['FECHA REESTRUCTURACION'])
df['reestruct_t1'] = False

df = df.merge(
    ree[['FOLADE','NEGOCIO','FECHA_REESTRUCTURACION']],
    how='left',
    on=['FOLADE','NEGOCIO']
)

df.loc[
    (df['FECHA_REESTRUCTURACION'] > df['FECHA_CORTE_BASE']) &
    (df['FECHA_REESTRUCTURACION'] <= df['fin_mes_t1']),
    'reestruct_t1'
] = True


# ===============================
# VARIABLE OBJETIVO
# ===============================

df['Y_malo'] = (
    (df['mora_t1'] > df['mora_t']) |
    (df['mora_t1'] > 120) |
    df['castigo_t1'] |
    df['reestruct_t1']
).astype(int)

df = df.dropna(subset=['mora_t1'])


# ===============================
# SEGMENTACIÓN POR TRAMO
# ===============================

def tramo(m):
    if m <= 30:
        return 'T1_0_30'
    elif m <= 60:
        return 'T2_31_60'
    elif m <= 120:
        return 'T3_61_120'
    else:
        return 'T4_120+'

df['TRAMO'] = df['mora_t'].apply(tramo)

In [ ]:
# ===============================
# FEATURES TEMPORALES
# ===============================

df['mora_t_1'] = df.groupby(['FOLADE','NEGOCIO'])['mora_t'].shift(1)
df['mora_t_2'] = df.groupby(['FOLADE','NEGOCIO'])['mora_t'].shift(2)

df['delta_mora_1'] = df['mora_t'] - df['mora_t_1']
df['delta_mora_2'] = df['mora_t_1'] - df['mora_t_2']

# Meses consecutivos en mora
df['meses_en_mora'] = (df['mora_t'] > 0).astype(int)

df['meses_en_mora'] = (
    df.groupby(['FOLADE','NEGOCIO'])['meses_en_mora']
      .apply(lambda x: x * (x.groupby((x == 0).cumsum()).cumcount() + 1))
      .reset_index(level=[0,1], drop=True)
)

# Ventanas móviles
df['mora_max_3m'] = (
    df.groupby(['FOLADE','NEGOCIO'])['mora_t']
      .rolling(window=3, min_periods=1)
      .max()
      .reset_index(level=[0,1], drop=True)
)

df['mora_mean_3m'] = (
    df.groupby(['FOLADE','NEGOCIO'])['mora_t']
      .rolling(window=3, min_periods=1)
      .mean()
      .reset_index(level=[0,1], drop=True)
)


# ===============================
# VARIABLES FINALES
# ===============================

features = [
    'SALDO TOTAL','SALDO CAPITAL','VALOR CUOTA','ratio_pago_saldo',
    'num_pagos','num_gestiones','contactabilidad',
    'ANTIGUEDAD ANIOS','SALARIO','Edad',
    'mora_t_1','mora_t_2','delta_mora_1','delta_mora_2',
    'meses_en_mora','mora_max_3m','mora_mean_3m'
]

df.columns = df.columns.str.replace(' ', '_')

features = [f.replace(' ', '_') for f in features]

df = df.loc[:, ~df.columns.duplicated(keep='last')]

df[features] = df[features].fillna(0)

tramos = ['T1_0_30', 'T2_31_60', 'T3_61_120', 'T4_120+']

In [ ]:
# ===============================
# MODELADO POR TRAMO (TIME SPLIT)
# ===============================

tramos = ['T1_0_30','T2_31_60','T3_61_120','T4_120+']

modelos = {}
metricas = {}
scorings = {}

for tramo_modelo in tramos:
    print(f' Tramo {tramo_modelo}')

    data = df[df['TRAMO'] == tramo_modelo].copy()
    data = data.sort_values('FECHA_CORTE_BASE')

    # Validación mínima
    if data['Y_malo'].nunique() < 2 or len(data) < 500:
        print(f'Tramo {tramo_modelo}: datos insuficientes')
        continue

    # Split temporal por meses
    meses = (
        data['FECHA_CORTE_BASE']
        .dt.to_period('M')
        .drop_duplicates()
        .sort_values()
    )

    cut_idx = int(len(meses) * 0.7)
    mes_corte = meses.iloc[cut_idx]
    fecha_corte = mes_corte.to_timestamp('M')

    train = data[data['FECHA_CORTE_BASE'] <= fecha_corte]
    test  = data[data['FECHA_CORTE_BASE'] > fecha_corte]

    #print('TRAIN:', train['FECHA_CORTE_BASE'].min(), train['FECHA_CORTE_BASE'].max())
    #print('TEST :', test['FECHA_CORTE_BASE'].min(), test['FECHA_CORTE_BASE'].max())

    X_train = train[features]
    y_train = train['Y_malo']

    X_test = test[features]
    y_test = test['Y_malo']

    # Modelo
    model = LGBMClassifier(
        n_estimators=500,
        max_depth=4,
        min_child_samples=50,
        learning_rate=0.05,
        class_weight='balanced',
        random_state=42,
        verbose=-1
    )

    model.fit(X_train, y_train)

    # Evaluación
    y_pred = model.predict_proba(X_test)[:, 1]
    y_pred_class = (y_pred >= 0.5).astype(int)
    # Usar 0.5 puede ser arbitrario, un thereshold óptimo puede mejorar las métricas , sin embargo para este caso no lo hicieron.
    auc = roc_auc_score(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    ks = np.max(tpr - fpr)
    cm = confusion_matrix(y_test, y_pred_class)

    print(f'AUC = {auc:.3f}')
    print('Matriz de Confusión:')
    print(cm)
    print('\nReporte de Clasificación:')
    print(classification_report(y_test, y_pred_class))

    # Guardar
    modelos[tramo_modelo] = model
    metricas[tramo_modelo] = {
        'AUC': auc,
        'KS': ks,
        'fecha_corte': fecha_corte
    }

    # Scoring por FOLADE–MES
    scoring = test[
        ['FOLADE', 'NEGOCIO', 'FECHA_CORTE_BASE', 'Y_malo']
    ].copy()

    scoring = scoring.reset_index(drop=True)
    scoring['score'] = y_pred
    scoring['pred_clase'] = (scoring['score'] >= 0.5).astype(int)

    scoring = scoring.sort_values(['FOLADE', 'FECHA_CORTE_BASE'])

    scorings[tramo_modelo] = scoring

In [ ]:
# ROC
resumen = (
    pd.DataFrame(metricas)
    .T
    .reset_index()
    .rename(columns={'index': 'TRAMO'})
)

print(resumen)

import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))

for tramo in scorings:
    data = scorings[tramo]
    fpr, tpr, _ = roc_curve(data['Y_malo'], data['score'])
    auc = metricas[tramo]['AUC']
    plt.plot(fpr, tpr, label=f'{tramo} (AUC={auc:.3f})')

plt.plot([0,1], [0,1], '--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC por Tramo')
plt.legend()
plt.grid(True)
plt.show()
